In [1]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import confusion_matrix, accuracy_score


In [2]:
df = pd.read_csv('news.csv')

In [3]:
df.shape

(6335, 4)

In [4]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
labels = df.label

In [6]:
labels.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [7]:
pac = PassiveAggressiveClassifier(max_iter = 50)

In [8]:
df = df[['text', 'label']]

In [9]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df.label = le.fit_transform(df.label)

In [10]:
df.head()

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",0
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,0
2,U.S. Secretary of State John F. Kerry said Mon...,1
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",0
4,It's primary day in New York and front-runners...,1


In [11]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stp_words = stopwords.words('english')
from nltk.stem import PorterStemmer

In [12]:
regex = re.compile('[^a-zA-Z]')
pst = PorterStemmer()

In [13]:
li = []
for sentence in df.text:
    tokens = word_tokenize(sentence)
    temp_li = []
    for word in tokens:
        if (word not in stp_words) & (len(word) > 2):
            word = pst.stem(word)
            word = regex.sub('',word)
            temp_li.append(word)
    temp_li = ' '.join(temp_li)
    li.append(temp_li)
            

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(stop_words = 'english', max_df = 0.7)


In [15]:
x = vect.fit_transform(li)

In [16]:
x = pd.DataFrame(x.toarray(), columns = vect.get_feature_names())


In [17]:
y = df['label']

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y, random_state = 42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [ ]:
model.fit(x_train, y_train)

In [ ]:
pred = model.predict(x_test)

In [ ]:
accuracy_score(y_test, pred)

In [ ]:
pac.fit(x_train, y_train)
pred = pac.predict(x_test)
accuracy_score(y_test, pred)

In [ ]:
confusion_matrix(y_test, pred)